In [1]:
#import needed librar
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import json
import psycopg2
import os
import re

In [4]:
def get_connection():
    
    with open('../db_connection/db_config_2.json') as my_file:
        db_conf = json.load(my_file)
    conn_str = "host={} dbname={} user={} password={}".format(
                db_conf['host'], db_conf['database'], 
                db_conf['user'], db_conf['passw'])
    try:
        if not hasattr(get_connection, "conn"):
            print('new')
            get_connection.conn = psycopg2.connect(conn_str)
            return get_connection.conn
        else:
            print('same')
            return get_connection.conn
        return conn
    except('error'):
        return 'error in connecting to database'


In [7]:
get_connection()

same


<connection object at 0x7f9dd0d887a0; dsn: 'host=41.89.93.180 password=xxx user=dsl dbname=mohdsl', closed: 0>

In [17]:
def get_indicator_values(facility_id, indicator_id, period_type, year ):
    conn = get_connection()
    
    #get the indicator details
    indicator = pd.read_sql( (''' select * from dim_dhis_indicator where indicatorid =  '%s' '''%(indicator_id)), conn).loc[0]
    
    #get all period values in that year and type  
     
    periods = pd.read_sql( "SELECT * FROM dim_dhis_period WHERE yearmonth @@ to_tsquery('%s') AND periodtypeid = '%s'" %(year,period_type) , con=conn)
    periods['year'] = periods['yearmonth'].str.split(' ').str[1]
    periods['month'] = periods['yearmonth'].str.split(' ').str[2]
    
    #get the facility selected    
    facility = pd.read_sql(''' SELECT  a.organisationunitid, a.code, b.name
                FROM dim_dhis_organisationunit a , facilities_facility b
                WHERE a.code = CAST(b.code as VarChar) AND b.id = '%s' ''' %(facility_id),conn)
    
    results = []
    
    #define the pattern for dataelements-categorycombo and extract them
    pattern = "[a-zA-Z0-9]+\.[a-zA-Z0-9]+|[a-zA-Z0-9]+"
    numerator_dataelements = re.findall(pattern,indicator['numerator'])
    denominator_dataelements = re.findall(pattern,indicator['denominator'])

    #iterate through all periods
    for col, period in periods.iterrows():
        numerator = indicator['numerator'] 
        
        #iterate through dataelements 
        for datelement_category_combo in numerator_dataelements:           
            #extract the data element by searching for datalement. and replace with the id
            datelement= None
            
            # check if the dataelement has a category combo
            combo = re.findall("\.[a-zA-Z0-9]+", datelement_category_combo)
            if combo:
                #remove . in category combo expression and get uid
                combo = re.sub('\.','', combo[0])
                
                
                
                query =  '''SELECT * 
                            FROM dim_dhis_categoryoptioncombo
                            WHERE uid = '%s'
                        ''' %(category_option_combo_id)
                all_category_option_combos =  pd.read_sql(query, con=conn)
                
                
                combo_uid = get_category_option_combo_by_uid(combo).loc[0]['categoryoptioncomboid']
                dataelement = re.sub('\.','' , re.findall("[a-zA-Z0-9]+\.", datelement_category_combo)[0]) 
            else:
                combo_uid = None
                dataelement = re.findall("[a-zA-Z0-9]+", datelement_category_combo)[0]

            value = get_facility_dataelement_datavalues(dataelement, facility['organisationunitid'], period['periodid'], combo_uid)

            if value.empty:            
                numerator = re.sub( "#{" + datelement_category_combo +"}" ,'0',numerator)                
            else:
                numerator = re.sub("#{" + datelement_category_combo +"}" ,value.loc[0]['value'],numerator)                
        numerator = float(eval(numerator))
        
        
        denominator = indicator['denominator']
        for datelement_category_combo in denominator_dataelements:                       
            datelement = None
            
            combo = re.findall("\.[a-zA-Z0-9]+", datelement_category_combo)
            if combo:
                combo = re.sub('\.','', combo[0])
                combo_uid = get_category_option_combo_by_uid(combo).loc[0]['categoryoptioncomboid']
                dataelement = re.sub('\.','' , re.findall("[a-zA-Z0-9]+\.", datelement_category_combo)[0]) 
            else:
                combo_uid = None
                dataelement = re.findall("[a-zA-Z0-9]+", datelement_category_combo)[0]

            value = get_facility_dataelement_datavalues(dataelement, facility['organisationunitid'], period['periodid'], combo_uid)
            if value.empty:      
                denominator = re.sub( "#{" + datelement_category_combo +"}" ,'1',denominator)                
            else:
                denominator = re.sub("#{" + datelement_category_combo +"}" ,value.loc[0]['value'],denominator)                
        denominator =  float(eval(denominator))
        indicator_value = {
            'indicator_id': indicator['indicatorid'],
            'value': (numerator/denominator),
            'year' : period['year'],
            'month': period['month']
        }
        
        results.append(indicator_value)
    
    return results
        

In [19]:
get_indicator_values('2e21b2de-7c5b-4db7-bb1d-26930af9274d',52910,5,2014)

same


NameError: global name 'get_category_option_combo_by_uid' is not defined